In [62]:
ROOT = "/Users/wubai.zhou/Downloads/models/"
GRAPH_FILE = ROOT + 'frozen_inference_graph.pb'

input_video = ROOT + 'carla_tl_loop.mp4'
output_video = ROOT + 'carla_tl_loop_labeled.mp4'
#input_video = 'test_vid_jupyter/carla_tl_loop.mp4'
#output_video = 'test_vid_jupyter/result_carla_tl_loop_ssd_mobilenet_v1_real_data.mp4'
#input_video = 'test_vid_jupyter/carla_just_tl.mp4'
#output_video = 'test_vid_jupyter/result_carla_just_tl_ssd_mobilenet_v1_real_data.mp4'

NUM_CLASSES = 4
category_index = {
    1: {'id': 1, 'name': 'Green'},
    2: {'id': 2, 'name': 'Red'},
    3: {'id': 3, 'name': 'Yellow'},
    4: {'id': 4, 'name': 'off'},
    10: {'id': 10, 'name': 'unkown'},
}
GRAPH_FILE

'/Users/wubai.zhou/Downloads/models/frozen_inference_graph.pb'

In [63]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
from PIL import ImageFont
import time
from scipy.stats import norm

In [64]:
# Colors (one for each class)
cmap = ImageColor.colormap
print("Number of colors =", len(cmap))
COLOR_LIST = sorted([c for c in cmap.keys()])

#
# Utility funcs
#

def filter_boxes(min_score, boxes, scores, classes, class_filter):
    """Return boxes with a confidence >= `min_score`"""
    n = len(classes)
    idxs = []
    for i in range(n):
        if scores[i] >= min_score: # and classes[i]==class_filter:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes

def to_image_coords(boxes, height, width):
    """
    The original box coordinate output is normalized, i.e [0, 1].
    
    This converts it back to the original coordinate based on the image
    size.
    """
    box_coords = np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] * height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width
    
    return box_coords

def draw_boxes(image, boxes, scores, classes, thickness=4):
    """Draw bounding boxes on the image"""
    draw = ImageDraw.Draw(image)
    for i in range(len(boxes)):
        bot, left, top, right = boxes[i, ...]
        class_id = int(classes[i])
        color = COLOR_LIST[class_id]
        
        class_name = category_index[classes[i]]['name']
        disp_txt = "%s : %f" % (class_name,scores[i])
        
        draw.line([(left, top), (left, bot), (right, bot), (right, top), (left, top)], width=thickness, fill=color)
        draw.text((left, top), disp_txt)

def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph

Number of colors = 148


In [65]:
print(GRAPH_FILE)
detection_graph  = load_graph(GRAPH_FILE)

# The input placeholder for the image.
# `get_tensor_by_name` returns the Tensor with the associated name in the Graph.
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

# The classification of the object (integer id).
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

/Users/wubai.zhou/Downloads/models/frozen_inference_graph.pb


In [66]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [67]:
clip = VideoFileClip(input_video)

In [68]:
def pipeline(img):
    draw_img = Image.fromarray(img)
    boxes, scores, classes = sess.run([detection_boxes, detection_scores, detection_classes], feed_dict={image_tensor: np.expand_dims(img, 0)})
    # Remove unnecessary dimensions
    boxes = np.squeeze(boxes)
    scores = np.squeeze(scores)
    classes = np.squeeze(classes).astype(np.int32)

    confidence_cutoff = 0.5
    class_filter = 10
    # Filter boxes with a confidence score less than `confidence_cutoff`
    boxes, scores, classes = filter_boxes(confidence_cutoff, boxes, scores, classes, class_filter)

    # The current box coordinates are normalized to a range between 0 and 1.
    # This converts the coordinates actual location on the image.
    width, height = draw_img.size
    box_coords = to_image_coords(boxes, height, width)

    # Each class with be represented by a differently colored box
    draw_boxes(draw_img, box_coords, scores, classes)
    return np.array(draw_img)

In [69]:
with tf.Session(graph=detection_graph) as sess:
    image_tensor = sess.graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = sess.graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = sess.graph.get_tensor_by_name('detection_scores:0')
    detection_classes = sess.graph.get_tensor_by_name('detection_classes:0')
    
    new_clip = clip.fl_image(pipeline)
    
    # write to file
    new_clip.write_videofile(output_video)

[MoviePy] >>>> Building video /Users/wubai.zhou/Downloads/models/carla_tl_loop_labeled.mp4
[MoviePy] Writing video /Users/wubai.zhou/Downloads/models/carla_tl_loop_labeled.mp4


100%|█████████▉| 1151/1152 [09:46<00:00,  2.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /Users/wubai.zhou/Downloads/models/carla_tl_loop_labeled.mp4 



In [70]:
HTML("""
<video width="960" height="600" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(output_video))

In [12]:
import yaml
file_dir = "/Users/wubai.zhou/Downloads/dataset-sdcnd-capstone/data/sim_training_data/"
%matplotlib inline  

In [14]:
anno_file = file_dir + "sim_data_annotations.yaml"
annotations = yaml.load(open(anno_file, 'r'))

In [10]:
annotations[0]

{'annotations': [{'class': 'Green',
   'x_width': 52.65248226950354,
   'xmin': 130.4964539007092,
   'y_height': 119.60283687943263,
   'ymin': 289.36170212765956},
  {'class': 'Green',
   'x_width': 50.156028368794296,
   'xmin': 375.60283687943263,
   'y_height': 121.87234042553195,
   'ymin': 293.90070921985813},
  {'class': 'Green',
   'x_width': 53.33333333333326,
   'xmin': 623.6595744680851,
   'y_height': 119.82978723404256,
   'ymin': 297.7588652482269}],
 'class': 'image',
 'filename': 'sim_data_capture/left0003.jpg'}

In [41]:
import numpy as np
color_to_id = {"Green": 1, "Red": 2, "Yellow": 3}
def extract_info(annos):
    boxes, classes, scores = [], [], []
    for anno in annos['annotations']:
        classes.append(color_to_id[anno['class']])
        ymin = anno['ymin']
        xmin = anno['xmin']
        ymax = ymin + anno['y_height']
        xmax = xmin + anno['x_width']
        boxes.append((ymin, xmin, ymax, xmax))
        scores.append(1)
    return np.array(boxes), np.array(classes), np.array(scores)

In [52]:
for anno in annotations:
    img = Image.open(file_dir + anno['filename'])
    boxes, classes, scores = extract_info(anno)
    draw_boxes(img, boxes=boxes, scores=scores, classes=classes)
    img.save(file_dir + 'combine/' + anno['filename'].split('/')[-1])

In [13]:
img = Image.open(file_dir + annotations[0]['filename'])
img.show()

In [43]:
boxes, classes, scores = extract_info(annotations[0])
boxes

array([[289.36170213, 130.4964539 , 408.96453901, 183.14893617],
       [293.90070922, 375.60283688, 415.77304965, 425.75886525],
       [297.75886525, 623.65957447, 417.58865248, 676.9929078 ]])

In [45]:
draw_boxes(img, boxes=boxes, scores=scores, classes=classes)

In [51]:
img.save(file_dir + 'combine/' + annotations[0]['filename'].split('/')[-1])

In [60]:
import os

In [61]:
os.path.join("hdfs:///user/wubai.zhou/ltv/postfix", 'x', 'y')

'hdfs:///user/wubai.zhou/ltv/postfix/x/y'